Il metodo scelto per l'assignment 1 è stato il metodo di Zhang [(Link alla pagina wikipedia)](https://en.wikipedia.org/wiki/Camera_resectioning#:~:text=Zhang%27s%20method%5Bedit%5D) 

I passaggi da effettuare sono
- Corrispondenza punti 2D e 3D
- Trovare la matrice P
- Usare fattorizzazione RQ (Variante della fattorizzazione QR)
- Trovare K che è la matrice R della fattorizzazione precedente
- Mentre la matrice R di rotazione (da non confondere con la R di RQ) sarà la matrice Q di RQ
- Trovare t vettore colonna della traslazione

Import di tutte le librerie necessarie
Usiamo numpy per la gestione delle matrici generate dalle immagini e math per fare i calcoli, glob ci aiuta a raccogliere tutti i file di un tipo 

In [3]:
import numpy as np
import math
import cv2
import glob
import os
from sympy import symbols, solve, simplify, Matrix, transpose, init_printing, pprint


ModuleNotFoundError: No module named 'numpy'

Definiamo la dimensioni della scacchiera e i criteri per fermare l'iterazione della rifinitura dei contorni e dei punti interni della scacchiera

In [22]:
scacchiera = (7,9)
criteri = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Definiamo quindi degli array per i punti nello spazio e nel piano ed inizializziamo i punti nello spazio a 0, 

In [6]:
punti3d  = []
punti2d = [] 
oggetto3d = np.zeros((1, scacchiera[0] * scacchiera[1], 3), np.float32)
oggetto3d[0,:,:2] = np.mgrid[0:scacchiera[0], 0:scacchiera[1]].T.reshape(-1, 2)

errori = []

Raccogliamo tutte le immagini di tipo jpg presenti nella directory immagini

In [13]:
immagini = glob.glob('datiLaboratorio\checkboard\*.jpeg')
print (len(immagini))

40


Creo la funzione custom calibrate camera

In [20]:
def calibra():
    alfa            = symbols ('alfa')
    beta            = symbols ('beta')
    gamma           = symbols ('gamma')
    u0              = symbols ('u0')
    v0              = symbols ('v0')
    A               = Matrix ([[alfa, gamma, u0], [0 , beta, v0 ], [0, 0 , 1]])
    #B              = simplify((transpose(A.inv()) * A.inv()).xreplace( {alfa:10 , beta:16 , gamma:1 , u0:182 , v0:122}))
    B               = simplify(transpose(A.inv()) * A.inv())
    b               = Matrix([B[0,0],B[0,1],B[1,1],B[0,2],B[1,2],B[2,2] ])
    b               = transpose(b)
    r11, r12, t1    = symbols ('r11 r12 t1')
    r21, r22, t2    = symbols ('r21 r22 t2')
    r31, r32, t3    = symbols ('r31 r32 t3')
    H               = A * Matrix ([[r11, r12, t1], [r21 , r22, t2 ], [r31, r32 , t3]])
    h1              = H[:,1]
    h2              = H[:,2]
    h3              = H[:,3]

    
    pprint (b)
calibra()

⎡                    2    2                      2                            
⎢  1      -γ     alfa  + γ   -β⋅u₀ + γ⋅v₀  - alfa ⋅v₀ + γ⋅(β⋅u₀ - γ⋅v₀)      v
⎢─────  ───────  ──────────  ────────────  ────────────────────────────  1 + ─
⎢    2      2         2  2         2                     2  2                 
⎣alfa   alfa ⋅β   alfa ⋅β      alfa ⋅β               alfa ⋅β                  

 2                2⎤
₀    (β⋅u₀ - γ⋅v₀) ⎥
── + ──────────────⎥
 2          2  2   ⎥
β       alfa ⋅β    ⎦


Definisco un doppio ciclo for, il primo serve per ciclare in un range 0,3 estremi inclusi per dividere il set di immagini in 4 per fare cross validation, il set di validazione sarà dato dal vettore tagliato dalla posizione iesima alla i+1 esima



In [25]:
for i in range (0,4):
    multiply = int(len(immagini)/4)
    a = int(multiply * i)
    b = int(multiply * i +1)
    validation = immagini[a : b]
    training = [i for i in immagini if i not in validation]
    for im in training:
        image = cv2.imread(im)
        scalagrigi = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ret, angoli = cv2.findChessboardCorners(scalagrigi, scacchiera, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret:
            punti3d.append(oggetto3d)
            angoli2d = cv2.cornerSubPix(scalagrigi, angoli, (11,11),(-1,-1), criteri)
            punti2d.append(angoli2d)

            immagine = cv2.drawChessboardCorners(image, scacchiera, angoli2d, ret)

        cv2.imshow('immagine', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    #errori[i] = cv2.calibrateCamera() - calibra()
 

datiLaboratorio\checkboard\test-121515.jpeg
[[[ 41  22  17]
  [ 41  22  17]
  [ 61  42  37]
  ...
  [ 73  55  48]
  [ 72  54  47]
  [ 71  53  46]]

 [[177 161 155]
  [195 179 173]
  [204 188 182]
  ...
  [ 70  52  45]
  [ 72  54  47]
  [ 74  56  49]]

 [[180 168 162]
  [165 153 147]
  [149 135 129]
  ...
  [ 74  56  49]
  [ 72  54  47]
  [ 70  52  45]]

 ...

 [[ 25  19  20]
  [ 25  19  20]
  [ 25  19  20]
  ...
  [ 62  50  40]
  [103  91  81]
  [122 110 100]]

 [[ 25  19  20]
  [ 25  19  20]
  [ 25  19  20]
  ...
  [ 67  55  45]
  [ 64  52  42]
  [118 106  96]]

 [[ 25  19  20]
  [ 25  19  20]
  [ 25  19  20]
  ...
  [ 65  53  43]
  [ 56  44  34]
  [ 79  67  57]]]
datiLaboratorio\checkboard\test-121516.jpeg
[[[ 41  22  17]
  [ 41  22  17]
  [ 61  42  37]
  ...
  [ 73  55  48]
  [ 72  54  47]
  [ 71  53  46]]

 [[177 161 155]
  [195 179 173]
  [204 188 182]
  ...
  [ 70  52  45]
  [ 72  54  47]
  [ 74  56  49]]

 [[180 168 162]
  [165 153 147]
  [149 135 129]
  ...
  [ 74  56  49]
  [ 